In [1]:
import datetime
import csv
import os

In [2]:
import random
import numpy as np

In [3]:
import pandas as pd


In [4]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot


In [5]:
path = 'data/states.csv'
DF = pd.read_csv(path)
DF['date'] = pd.to_datetime(DF['date'])
DF.head()


,date,state,cases,deaths
0,2020-01-22,Washington,0,0
1,2020-01-23,Washington,0,0
2,2020-01-24,Washington,0,0
3,2020-01-25,Illinois,0,0
4,2020-01-25,Washington,0,0


In [6]:
#texas opened up April 30
df_first_texas = DF[(DF['date'] > datetime.datetime(2020, 4, 15)) &(DF['date'] <= datetime.datetime(2020,4,30))
            & (DF['state'] == 'Texas')]
df_second_texas = DF[(DF['date'] > datetime.datetime(2020, 4, 30)) &(DF['date'] <= datetime.datetime(2020,5,14))
            & (DF['state'] == 'Texas')]
df_third_texas = DF[(DF['date'] > datetime.datetime(2020, 5, 14)) &(DF['date'] <= datetime.datetime(2020,6,1))
            & (DF['state'] == 'Texas')]

In [12]:
def resample_two_samples(sample1, sample2, num_iterations = 1000):
    both = sample1 + sample2
    mean1 = []
    mean2 = []
    for i in range(num_iterations):
        random.shuffle(both) 
        new_1 = both[0:len(sample1)] 
        new_2 = both[len(sample1):] 
        mean1.append(np.mean(new_1)) 
        mean2.append(np.mean(new_2)) 
    return mean1, mean2
def combine_resamples(sample1, sample2, resample1, resample2):
    diff1 = [np.mean(sample1 )- x for x in resample1]
    diff2 = [np.mean(sample2)- x for x in resample2]
    both = np.array(diff1) - np.array(diff2) 
    return both

def get_p_value(l, v = 0):
    p = 1 - len([x for x in l if x > v])/len(l)
    return p

def do_hist(x, bins=10, width = 250, height = 250, title = None):
  hist, edges = np.histogram(x, bins = bins, density=True)
  p_hist = figure(width = width, height = height, title = title)
  p_hist.quad(top = hist, bottom=0, left=edges[:-1], right=edges[1:], alpha = .4)
  return p_hist

In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
values_death = [['', 'Pre Open', 'Second Week', 'Difference', 'Significant', 'Third Week', 
         'Difference', 'Significant']]
values_cases = [['', 'Pre Open',  'Second Week', 'Difference', 'Significant', 'Third Week', 
         'Difference', 'Significant']]
graphs = []

In [10]:
# get the means
f = np.mean(df_first_texas['deaths'])
s = np.mean(df_second_texas['deaths'])
t = np.mean(df_third_texas['deaths'])
print(f, s , t)


28.866666666666667 30.571428571428573 24.38888888888889


In [11]:
a,b = resample_two_samples(df_first_texas['deaths'].tolist(), df_second_texas['deaths'].tolist())
l = combine_resamples(df_second_texas['deaths'].tolist(),
            df_first_texas['deaths'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Deaths Texas 1' ))
get_p_value(l)


0.32899999999999996

In [12]:
a,b = resample_two_samples(df_first_texas['deaths'].tolist(), df_third_texas['deaths'].tolist())
l = combine_resamples(df_third_texas['deaths'].tolist(),
             df_first_texas['deaths'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Deaths Texas 2' ))
get_p_value(l)


0.855

In [13]:
values_death.append(['Texas', round(f,1), round(s,1), round(s-f,1), 'No', round(t,1), round(t-f,1), 'No'])

In [14]:
#for cases
f = np.mean(df_first_texas['cases'])
s = np.mean(df_second_texas['cases'])
t = np.mean(df_third_texas['cases'])
print(f, s, t)
a,b = resample_two_samples(df_first_texas['cases'].tolist(), df_second_texas['cases'].tolist())
l = combine_resamples(df_second_texas['cases'].tolist(),
             df_first_texas['cases'].tolist(),
                   b,a
                     )
graphs.append(do_hist(l, title = 'Cases Texas 1' ))
get_p_value(l)

864.4 1149.7857142857142 1161.7777777777778


0.0

In [15]:
a,b = resample_two_samples(df_first_texas['cases'].tolist(), df_third_texas['cases'].tolist())
l = combine_resamples(df_third_texas['cases'].tolist(),
             df_first_texas['cases'].tolist(),
                   b,a
                )
graphs.append(do_hist(l, title = 'Cases Texas 2' ))
get_p_value(l)


0.0010000000000000009

In [16]:
"""Conclusion: No significance in difference between deaths, but a strong difference between last two weeks
and pre-opening period"""

'Conclusion: No significance in difference between deaths, but a strong difference between last two weeks\nand pre-opening period'

In [17]:
values_cases.append(['Texas', round(f,1), round(s,1), round(s-f,1), 'Yes', round(t,1), round(t-f,1), 'Yes'])

In [18]:
#Arizona opened up May 15
df_first_arizona = DF[(DF['date'] > datetime.datetime(2020, 5, 1)) &(DF['date'] <= datetime.datetime(2020,5,15))
            & (DF['state'] == 'Arizona')]
df_second_arizona = DF[(DF['date'] > datetime.datetime(2020, 5, 15)) &(DF['date'] <= datetime.datetime(2020,5,30))
            & (DF['state'] == 'Arizona')]


In [19]:
f = np.mean(df_first_arizona['deaths'])
s = np.mean(df_second_arizona['deaths'])
print(f,s)


22.928571428571427 16.8


In [20]:
a,b = resample_two_samples(df_first_arizona['deaths'].tolist(), df_second_arizona['deaths'].tolist())
l = combine_resamples(df_second_arizona['deaths'].tolist(),
             df_first_arizona['deaths'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Deaths Arizona 1' ))
get_p_value(l)


0.861

In [21]:
"""Conclusion: deaths declined, but not significantly"""

'Conclusion: deaths declined, but not significantly'

In [22]:
values_death.append(['Arizona', round(f,1), round(s,1), round(s-f,1), 'No', 'N/A', 'N/A', 'No'])

In [23]:
f = np.mean(df_first_arizona['cases'])
s = np.mean(df_second_arizona['cases'])
print(f,s)

371.92857142857144 405.73333333333335


In [24]:
a,b = resample_two_samples(df_first_arizona['cases'].tolist(), df_second_arizona['deaths'].tolist())
l = combine_resamples(df_second_arizona['cases'].tolist(),
             df_first_arizona['cases'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Cases Arizona 1' ))
graphs.append(None)
graphs.append(None)
get_p_value(l)

0.31399999999999995

In [25]:
values_cases.append(['Arizona', round(f,1), round(s,1), round(s-f,1), 'No', 'N/A', 'N/A', 'No'])

In [26]:
"""Concluslion: cases have risen, but not significantly"""

'Concluslion: cases have risen, but not significantly'

In [27]:
#Arkansas opened up May 6
df_first_arkansas = DF[(DF['date'] > datetime.datetime(2020, 4, 22)) &(DF['date'] <= datetime.datetime(2020,5,6))
            & (DF['state'] == 'Arkansas')]
df_second_arkansas = DF[(DF['date'] > datetime.datetime(2020, 5, 6)) &(DF['date'] <= datetime.datetime(2020,5,20))
            & (DF['state'] == 'Arkansas')]
df_third_arkansas = DF[(DF['date'] > datetime.datetime(2020, 5, 20)) &(DF['date'] <= datetime.datetime(2020,6,1))
            & (DF['state'] == 'Arkansas')]


In [28]:
f = np.mean(df_first_arkansas['deaths'])
s = np.mean(df_second_arkansas['deaths'])
t = np.mean(df_third_arkansas['deaths'])
print(f,s,t)

3.0714285714285716 1.4285714285714286 2.1666666666666665


In [29]:
a,b = resample_two_samples(df_first_arkansas['deaths'].tolist(), df_second_arkansas['deaths'].tolist())
l = combine_resamples(df_second_arkansas['deaths'].tolist(),
             df_first_arkansas['deaths'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Deaths Arkansas 1' ))
get_p_value(l)

0.989

In [30]:
a,b = resample_two_samples(df_first_arkansas['deaths'].tolist(), df_third_arkansas['deaths'].tolist())
l = combine_resamples(df_third_arkansas['deaths'].tolist(),
             df_third_arkansas['deaths'].tolist(),
                   a, b
                  )
graphs.append(do_hist(l, title = 'Deaths Arkansas 2' ))
get_p_value(l)

0.492

In [31]:
"""Conclusion: Deaths dropped significantly for the second week. Deaths rose for the third week, but not
significantly"""

'Conclusion: Deaths dropped significantly for the second week. Deaths rose for the third week, but not\nsignificantly'

In [32]:
values_death.append(['Arkansas', round(f,1), round(s,1), round(s-f,1), 'No', round(t,1), round(t-f,1), 'No'])

In [33]:
f = np.mean(df_first_arkansas['cases'])
s = np.mean(df_second_arkansas['cases'])
t = np.mean(df_third_arkansas['cases'])
print(f,s,t)

87.07142857142857 99.42857142857143 203.33333333333334


In [34]:
a,b = resample_two_samples(df_first_arkansas['cases'].tolist(), df_second_arkansas['cases'].tolist())
l = combine_resamples(df_second_arkansas['cases'].tolist(),
             df_first_arkansas['cases'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Cases Arkansas 1' ))
get_p_value(l)

0.30200000000000005

In [35]:
a,b = resample_two_samples(df_first_arkansas['cases'].tolist(), df_third_arkansas['cases'].tolist())
l = combine_resamples(df_third_arkansas['cases'].tolist(),
             df_first_arkansas['cases'].tolist(),
                   b,a
                  )
graphs.append(do_hist(l, title = 'Cases Arkansas 2' ))
get_p_value(l)

0.0010000000000000009

In [36]:
"""Conclusion: cases rose in the second week, but not significantly. Cases rose significantly in the third 
week."""

'Conclusion: cases rose in the second week, but not significantly. Cases rose significantly in the third \nweek.'

In [37]:
values_cases.append(['Arkansas', round(f,1), round(s,1), round(s-f,1), 'No', round(t,1), round(t-f,1), 'Yes'])

In [38]:
with open(os.path.join('data', 'states_diff.csv'), 'w') as write_obj:
    csv_writer = csv.writer(write_obj)
    csv_writer.writerows(values_death)

In [39]:
with open(os.path.join('data', 'states_diff_cases.csv'), 'w') as write_obj:
    csv_writer = csv.writer(write_obj)
    csv_writer.writerows(values_cases)

In [40]:
grid = gridplot(graphs, ncols = 4)
show(grid)


In [45]:
df_second_arizona = DF[(DF['date'] >=datetime.datetime(2020, 5, 26)) 
            & (DF['state'] == 'Arizona')]

df_first_arizona = DF[(DF['date'] < datetime.datetime(2020, 5, 26)) & (DF['date'] > datetime.datetime(2020,5,12))
            & (DF['state'] == 'Arizona')]

In [10]:
df_first_arizona.head()

,date,state,cases,deaths
4977,2020-06-02,Arizona,1127,25
5032,2020-06-03,Arizona,995,40
5087,2020-06-04,Arizona,573,17
5142,2020-06-05,Arizona,1551,16
5197,2020-06-06,Arizona,1148,30


In [46]:
a,b = resample_two_samples(df_first_arizona['cases'].tolist(), df_second_arizona['cases'].tolist())
l = combine_resamples(df_first_arizona['cases'].tolist(),
             df_second_arizona['cases'].tolist(),
                   b,a
                  )


In [21]:
output_notebook()

Loading BokehJS ...

In [48]:
pf = do_hist(df_first_arizona['cases'].tolist(), width = 350, height = 350)
ps = do_hist(df_second_arizona['cases'].tolist(), width = 350, height = 350)
p = do_hist(l, width = 350, height = 350)
grid = gridplot([pf, ps, p], ncols = 3)
show(grid)


In [64]:
df_second_arizona_ = DF[(DF['date'] >=datetime.datetime(2020, 5, 20)) 
            & (DF['state'] == 'Arizona')]
dates = df_second_arizona_['date']
dates = [datetime.datetime(x.year, x.month, x.day) for x in dates]
nums = df_second_arizona_['cases'].rolling(7).mean()
p = figure(x_axis_type = 'datetime', 
            plot_width = 350 , plot_height = 350)
p.vbar(x=dates, top=nums)
show(p)

In [75]:
len(nums)
f = nums[6:13].tolist()
s = nums[13:].tolist()


In [77]:
a,b = resample_two_samples(f, s)
l = combine_resamples(f, s,
                   a,b
                  )
p = do_hist(l)
show(p)